In [1]:
from openai import OpenAI
from pinecone import Pinecone
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
import os


c:\Users\anees\Desktop\Coding\llmrepo\train-LLAMA\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
load_dotenv()

True

In [3]:
pc = Pinecone(os.getenv("PINECONE_KEY"))

In [4]:
client = OpenAI(
    base_url='http://localhost:11434/v1',
    api_key='ollama'
)

In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [7]:
def embed_input(input_text):
    return model.encode(input_text).tolist()

In [23]:
query = "Who is Aneesh ?"
query_embedding = embed_input(query)
print(query_embedding)

[-0.0707567110657692, 0.06581093370914459, -0.045117661356925964, -0.00204282789491117, -0.10098598152399063, 0.040134962648153305, 0.1337764412164688, -0.04283968359231949, -0.002322412794455886, 0.08056382834911346, 0.024703649803996086, -0.12464013695716858, 0.01958180032670498, -0.03169500082731247, -0.09184026718139648, 0.05348376929759979, -0.011786093935370445, -0.11373955011367798, -0.021035127341747284, -0.12792934477329254, -0.11866891384124756, 0.02679341286420822, -0.005795611068606377, -0.014591677114367485, -0.05919959768652916, -0.03437770903110504, -0.042153581976890564, -0.037528567016124725, -0.012216174975037575, -0.05240365490317345, -0.008944762870669365, -0.009397557005286217, 0.045220136642456055, 0.04808253049850464, 0.022381354123353958, 0.10404099524021149, 0.08397827297449112, 0.06523140519857407, -0.06933518499135971, 0.04976838454604149, 0.007597016170620918, -0.039258264005184174, -0.00198427215218544, -0.045884184539318085, -0.013882546685636044, 0.023940

In [24]:
index = pc.Index("rag-resume-data")

In [10]:
def revelant_data(query_embedding):
    return index.query(vector=query_embedding, top_k=5, include_metadata=True)

In [25]:
context = revelant_data(query_embedding)
context

{'matches': [{'id': 'doc-1',
              'metadata': {'prefix': 'contact.name', 'text': 'Aneesh Patne'},
              'score': 0.640817344,
              'values': []},
             {'id': 'doc-5',
              'metadata': {'prefix': 'contact.leetcode',
                           'text': 'https://leetcode.com/aneeshpatne'},
              'score': 0.521952271,
              'values': []},
             {'id': 'doc-4',
              'metadata': {'prefix': 'contact.github',
                           'text': 'https://github.com/aneeshpatne'},
              'score': 0.479536831,
              'values': []},
             {'id': 'doc-3',
              'metadata': {'prefix': 'contact.linkedin',
                           'text': 'https://www.linkedin.com/in/aneeshpatne'},
              'score': 0.433732748,
              'values': []},
             {'id': 'doc-2',
              'metadata': {'prefix': 'contact.email',
                           'text': 'aneeshpatne12@gmail.com'},
          

In [26]:
word_limit = 100  # Adjust as needed

prompt = f'''
You are a professional Resume Assistant bot that communicates on behalf of the candidate, using the resume data provided via Retrieval-Augmented Generation (RAG). Your task is to answer questions, provide clarifications, or engage in dialogue with users (such as potential employers or recruiters) based on the resume context. Maintain a formal yet approachable tone suitable for professional interactions.

**Resume Context:**
{context}

**Instructions:**
1. Base all responses strictly on the resume context provided.
2. Focus only on information relevant to the query, and ignore any extraneous or unrelated details.
3. Tailor your communication for professional scenarios—whether responding to recruiter inquiries or discussing qualifications with a company.
4. Use clear, concise language to highlight the candidate's skills, experience, and achievements as they pertain to the question at hand.
5. Avoid fabricating details; if relevant information is missing, politely request more specifics.
6. Keep responses within {word_limit} words and format them in well-structured paragraphs.

**Interaction Scenario:**
{query}

**Output:**
Generate a response that addresses the query using only the relevant parts of the resume context. Emphasize applicable skills, experiences, and achievements, and structure your answer professionally. Highlight measurable results and specific skills where possible.

**Fallback:**
If the resume context lacks sufficient information for a comprehensive response, reply with:
"I need more specific details to tailor the response effectively. Could you provide more information about [specific detail, e.g., the job role, required skills]?"
'''


In [28]:
developer_message = {
    "role": "developer",
    "content":(
        "You are a professional Resume Assistant bot that communicates on behalf of the candidate, "
        "using the resume data provided via Retrieval-Augmented Generation (RAG). Your task is "
        "to answer questions, provide clarifications, or engage in dialogue with users (such as "
        "potential employers or recruiters) based on the resume context. Maintain a formal yet "
        "approachable tone suitable for professional interactions."
    )
}

In [30]:
user_message = {
    "role": "user",
    "content": prompt
}

In [31]:
try:
    response_stream = client.chat.completions.create(
        model="mistral:latest",
        messages=[
            developer_message,
            user_message
        ],
        stream=True
    )

    for chunk in response_stream:
        if chunk.choices and len(chunk.choices) > 0:
            delta = chunk.choices[0].delta
            if delta and delta.content:
                print(delta.content, end="", flush=True)

except Exception as e:
    print(e)


Error code: 400 - {'error': {'message': 'invalid message content type: map[string]interface {}', 'type': 'invalid_request_error', 'param': None, 'code': None}}
